<a href="https://colab.research.google.com/github/praffuln/agentic-ai/blob/master/memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('helloworld')

helloworld


# Lesson 2: Baseline Email Assistant

This lesson builds an email assistant that:
- Classifies incoming messages (respond, ignore, notify)
- Drafts responses
- Schedules meetings

We'll start with a simple implementation - one that uses hard-coded rules to handle emails.

![Memory Course App](./img/memory_course_email.png)


### Setup Environments


In [ ]:
!pip install google-generativeai tiktoken chromadb langgraph langchain langchain_community langchainhub ipykernel langchain_groq sentence_transformers boto3 langchain_aws docx exceptions langchain_openai langchain-google-vertexai
!pip install -U langchain-google-vertexai
!pip install --upgrade python-docx


 ## Setup Keys For GOOGLE_API_KEY


In [6]:

 # Assigning value to variable
GEMINI_API_KEY=''
SERPER_API_KEY = ''



# setup environments


In [25]:
import os
os.environ['SERPER_API_KEY'] = SERPER_API_KEY
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = GEMINI_API_KEY




## setup LLM

In [30]:
import google.generativeai as genai
genai.configure(api_key=GEMINI_API_KEY)
llm = genai.GenerativeModel("gemini-1.5-flash")
response = llm.generate_content("Explain how AI works")
print(response.text)




Artificial intelligence (AI) is a broad field encompassing various techniques that enable computers to mimic human intelligence.  There's no single "how it works" answer, as different AI approaches utilize different methods. However, we can break down some core principles:

**1. Data is King:**  AI systems learn from data.  The more relevant and high-quality data you feed an AI, the better it will perform. This data can be anything from images and text to sensor readings and financial transactions.

**2. Algorithms are the Tools:**  Algorithms are sets of rules and statistical techniques that process the data. These algorithms allow the AI to identify patterns, make predictions, and learn from its mistakes.  Different types of AI use different algorithms:

* **Machine Learning (ML):**  This is a subset of AI where systems learn from data without explicit programming.  Instead of being explicitly programmed with rules, ML algorithms identify patterns in data and build models to make pre

## Setup a Profile, Prompt and Example Email

In [9]:
profile = {
    "name": "John",
    "full_name": "John Doe",
    "user_profile_background": "Senior software engineer leading a team of 5 developers",
}

In [10]:
prompt_instructions = {
    "triage_rules": {
        "ignore": "Marketing newsletters, spam emails, mass company announcements",
        "notify": "Team member out sick, build system notifications, project status updates",
        "respond": "Direct questions from team members, meeting requests, critical bug reports",
    },
    "agent_instructions": "Use these tools when appropriate to help manage John's tasks efficiently."
}

In [11]:
# Example incoming email
email = {
    "from": "Alice Smith <alice.smith@company.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Quick question about API documentation",
    "body": """
Hi John,

I was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?

Specifically, I'm looking at:
- /auth/refresh
- /auth/validate

Thanks!
Alice""",
}

## Define the first part of the agent - triage.

In [31]:
from pydantic import BaseModel, Field
from typing_extensions import TypedDict, Literal, Annotated
from langchain.chat_models import init_chat_model

In [32]:
class Router(BaseModel):
    """Analyze the unread email and route it according to its content."""

    reasoning: str = Field(
        description="Step-by-step reasoning behind the classification."
    )
    classification: Literal["ignore", "respond", "notify"] = Field(
        description="The classification of an email: 'ignore' for irrelevant emails, "
        "'notify' for important information that doesn't need a response, "
        "'respond' for emails that need a reply",
    )

In [33]:
llm_router = llm.with_structured_output(Router)


AttributeError: 'GenerativeModel' object has no attribute 'with_structured_output'

In [2]:
!pip install langchain_google_genai
from langchain_google_genai import GoogleGenerativeAI
from pydantic import BaseModel, Field
from typing_extensions import TypedDict, Literal, Annotated

class Router(BaseModel):
    """Analyze the unread email and route it according to its content."""

    reasoning: str = Field(
        description="Step-by-step reasoning behind the classification."
    )
    classification: Literal["ignore", "respond", "notify"] = Field(
        description="The classification of an email: 'ignore' for irrelevant emails, "
        "'notify' for important information that doesn't need a response, "
        "'respond' for emails that need a reply",
    )

# Wrap the Google Generative AI model with LangChain's wrapper
# Use `model` instead of `model_name` to specify the model
llm = GoogleGenerativeAI(model="models/gemini-1.5-flash", temperature=0) # temperature=0 for more deterministic outputs


# Now you can use with_structured_output
llm_router = llm.with_structured_output(Router)


NotImplementedError: 

In [ ]:
from prompts import triage_system_prompt, triage_user_prompt